> Algumas questões foram alteradas depois do envio na Udemy

##### Parte 1 > Realizando import's e iniciando pyspark

In [36]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
from pyspark.sql.types import StringType
import random
from pyspark.sql.functions import *

In [37]:
# Nesta etapa, adicoine código para ler o arquivo nomes_aleatorios.txt através do comando spark.read.csv. 
# Carregue-o para dentro de um dataframe chamado df_nomes e, por fim, liste algumas linhas através do método show. Exemplo: df_nomes.show(5)

spark = SparkSession.builder.master("local[*]").appName("Tarefa").getOrCreate()
df_nomes = spark.read.csv('../Tarefa3/nomes_aleatorios.txt')
df_nomes.show(5)

+----------------+
|             _c0|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows



##### Parte 2 > Renomear coluna, imprimir o schema e mostrar 10 linhas

In [38]:
df_nomes = df_nomes.withColumnRenamed('_c0', 'Nomes')
df_nomes.printSchema()
df_nomes.show(10)


root
 |-- Nomes: string (nullable = true)

+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



##### Parte 3 > Adicionar nova coluna chamada Escolaridade

In [39]:
escolaridade = ['Fundamental', 'Médio', 'Superior']

df_nomes = df_nomes.withColumn('Escolaridade', udf(f=lambda: escolaridade[random.randint(0,2)],returnType=StringType())())

##### Parte 4 > Adicionar países da América do Sul


In [40]:
paises = ["Brasil", "Argentina", "Colômbia", "Peru", "Venezuela", "Chile", "Equador",
            "Bolívia", "Paraguai", "Uruguai", "Guiana", "Suriname", "Guiana Francesa"]

df_nomes = df_nomes.withColumn('Pais', udf(f=lambda: paises[random.randint(0,12)],returnType=StringType())())

##### Parte 5 > Adicione nova coluna chamada AnoNascimento

In [41]:
df_nomes = df_nomes.withColumn("AnoNascimento", expr("CAST(FLOOR(RAND() * (66) + 1945) AS INT)"))

##### Parte 6 > Usando o método select do dataframe (df_nomes), selecione as pessoas que nasceram neste século

In [42]:
df_select = df_nomes.select('*').where(df_nomes.AnoNascimento >= 2000)
df_select.show(10)

+-----------------+------------+---------------+-------------+
|            Nomes|Escolaridade|           Pais|AnoNascimento|
+-----------------+------------+---------------+-------------+
|      Joy Bennett| Fundamental|        Uruguai|         2003|
|     David Medina| Fundamental|          Chile|         2001|
|      Jessie Jean|    Superior|         Brasil|         2004|
|    Michael Agnew| Fundamental|        Uruguai|         2001|
|   Rosie Lovelady| Fundamental|Guiana Francesa|         2006|
|    Milton Dillon|    Superior|      Argentina|         2007|
|   Ashley Trosper| Fundamental|        Equador|         2004|
|      Rita Walter|    Superior|Guiana Francesa|         2009|
|George Fiorentino|    Superior|Guiana Francesa|         2006|
|   David Rathbone| Fundamental|       Paraguai|         2001|
+-----------------+------------+---------------+-------------+
only showing top 10 rows



##### Parte 7 > Usando Spark SQL repita o processo da Pergunta 6.

In [43]:
df_nomes.createOrReplaceTempView('pessoas')
spark.sql('SELECT * FROM pessoas WHERE AnoNascimento >= 2000').show(10)


+-----------------+------------+--------+-------------+
|            Nomes|Escolaridade|    Pais|AnoNascimento|
+-----------------+------------+--------+-------------+
|      Joy Bennett|       Médio| Uruguai|         2003|
|     David Medina| Fundamental| Bolívia|         2001|
|      Jessie Jean|       Médio|  Brasil|         2004|
|    Michael Agnew| Fundamental|  Guiana|         2001|
|   Rosie Lovelady|    Superior| Uruguai|         2006|
|    Milton Dillon|    Superior| Bolívia|         2007|
|   Ashley Trosper| Fundamental| Bolívia|         2004|
|      Rita Walter|    Superior|    Peru|         2009|
|George Fiorentino|       Médio|Paraguai|         2006|
|   David Rathbone|    Superior|   Chile|         2001|
+-----------------+------------+--------+-------------+
only showing top 10 rows



##### Parte 8 > Conte o número de pessoas que são da geração Millennials


In [44]:
m = df_nomes.select('*').where(df_nomes.AnoNascimento.between(1980, 1994)).count()
print(m)


2272027


##### Parte 9 > Repita o processo da Pergunta 8 utilizando Spark SQL

In [45]:
spark.sql('SELECT COUNT(*) as Contagem FROM pessoas WHERE AnoNascimento BETWEEN 1980 AND 1994').show()

+--------+
|Contagem|
+--------+
| 2272027|
+--------+



##### Parte 10 > Obtenha a quantidade de pessoas de cada país para uma das gerações

In [46]:
sqlContext = SQLContext(spark)

babys = sqlContext.sql("SELECT Pais, 'Baby Boomers' AS geracao, count(*) as Quantidade\
                        FROM pessoas\
                        WHERE AnoNascimento BETWEEN 1944 AND 1964\
                        GROUP BY Pais")

ger_x = sqlContext.sql("SELECT Pais, 'Geração X' AS geracao, count(*) as Quantidade\
                        FROM pessoas\
                        WHERE AnoNascimento BETWEEN 1965 AND 1979\
                        GROUP BY Pais\
                        ORDER BY Pais, Geracao, Quantidade")

millenials = sqlContext.sql("SELECT Pais, 'Millennials' AS geracao, count(*) as Quantidade\
                        FROM pessoas\
                        WHERE AnoNascimento BETWEEN 1980 AND 1994\
                        GROUP BY Pais\
                        ORDER BY Pais, Geracao, Quantidade")

ger_z = sqlContext.sql("SELECT Pais, 'Geração Z' AS geracao, count(*) as Quantidade\
                        FROM pessoas\
                        WHERE AnoNascimento BETWEEN 1995 AND 2015\
                        GROUP BY Pais\
                        ORDER BY Pais, Geracao, Quantidade")

In [47]:
babys.orderBy(asc("Pais"),"Geracao", asc("Quantidade")).show()

+---------------+------------+----------+
|           Pais|     geracao|Quantidade|
+---------------+------------+----------+
|      Argentina|Baby Boomers|    232697|
|        Bolívia|Baby Boomers|    233576|
|         Brasil|Baby Boomers|    233128|
|          Chile|Baby Boomers|    232986|
|       Colômbia|Baby Boomers|    233830|
|        Equador|Baby Boomers|    233852|
|         Guiana|Baby Boomers|    232434|
|Guiana Francesa|Baby Boomers|    232997|
|       Paraguai|Baby Boomers|    232409|
|           Peru|Baby Boomers|    233617|
|       Suriname|Baby Boomers|    233599|
|        Uruguai|Baby Boomers|    233436|
|      Venezuela|Baby Boomers|    233523|
+---------------+------------+----------+



In [48]:
ger_x.orderBy(asc("Pais"),"Geracao", asc("Quantidade")).show()

+---------------+---------+----------+
|           Pais|  geracao|Quantidade|
+---------------+---------+----------+
|      Argentina|Geração X|    174321|
|        Bolívia|Geração X|    174659|
|         Brasil|Geração X|    174572|
|          Chile|Geração X|    174274|
|       Colômbia|Geração X|    174189|
|        Equador|Geração X|    175149|
|         Guiana|Geração X|    175015|
|Guiana Francesa|Geração X|    174972|
|       Paraguai|Geração X|    174542|
|           Peru|Geração X|    174469|
|       Suriname|Geração X|    175049|
|        Uruguai|Geração X|    174464|
|      Venezuela|Geração X|    174610|
+---------------+---------+----------+



In [50]:
millenials.orderBy(asc("Pais"),"Geracao", asc("Quantidade")).show()

+---------------+-----------+----------+
|           Pais|    geracao|Quantidade|
+---------------+-----------+----------+
|      Argentina|Millennials|    175100|
|        Bolívia|Millennials|    175190|
|         Brasil|Millennials|    174415|
|          Chile|Millennials|    174004|
|       Colômbia|Millennials|    174388|
|        Equador|Millennials|    175131|
|         Guiana|Millennials|    175015|
|Guiana Francesa|Millennials|    175104|
|       Paraguai|Millennials|    174917|
|           Peru|Millennials|    174557|
|       Suriname|Millennials|    174986|
|        Uruguai|Millennials|    174635|
|      Venezuela|Millennials|    174585|
+---------------+-----------+----------+



In [ ]:
ger_z.orderBy(asc("Pais"),"Geracao", asc("Quantidade")).show()

+---------------+---------+----------+
|           Pais|  geracao|Quantidade|
+---------------+---------+----------+
|      Argentina|Geração Z|    186139|
|        Bolívia|Geração Z|    186381|
|         Brasil|Geração Z|    186041|
|          Chile|Geração Z|    186457|
|       Colômbia|Geração Z|    186738|
|        Equador|Geração Z|    185831|
|         Guiana|Geração Z|    185929|
|Guiana Francesa|Geração Z|    186967|
|       Paraguai|Geração Z|    186483|
|           Peru|Geração Z|    186311|
|       Suriname|Geração Z|    186412|
|        Uruguai|Geração Z|    186872|
|      Venezuela|Geração Z|    186380|
+---------------+---------+----------+

